In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update > /dev/null
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar > /dev/null

--2022-04-02 15:47:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.94MB/s    in 0.2s    

2022-04-02 15:47:51 (4.94 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
jdbc_url="jdbc:postgresql://dataviz.cvt76u8nruy2.us-east-1.rds.amazonaws.com/challenge_db"
df = spark.read.format("jdbc").option("url",jdbc_url).option("dbtable", "vine_table").option("user","").option("password","").load()

In [5]:
df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

In [9]:
df_filtered = df[df.total_votes > 20]
df_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|                Y|
|R274AKHXHYGIN6|          5|           87|        100|   N|     

In [12]:
div_df = df_filtered[(df_filtered.helpful_votes/df_filtered.total_votes) >= 0.5]
print(div_df.count())
div_df.show()

47808
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KDZ3I0FHXMZQ|          5|           24|         31|   N|                N|
|R3S1MK1A69LEFQ|          1|           27|         38|   N|                Y|
|R13LPKFZJRUA5A|          4|           36|         41|   N|                Y|
|R1563GUPPMS9F6|          5|           47|         49|   N|                Y|
|R18QIXAW1NSCSU|          2|           32|         41|   N|                Y|
|R2Y6O5QUGKMRFX|          4|           27|         35|   N|                N|
|R3USE2RT48YCAT|          1|           25|         28|   N|                N|
| RS3ULHXX7AI3R|          4|           22|         24|   N|                Y|
|R38841CPL87RGV|          4|           19|         21|   N|                Y|
|R20AK9R2LBRPMK|          1|           27|         34|   N

In [14]:
vine_program_df = div_df[div_df.verified_purchase == 'Y']
print(vine_program_df.count())
vine_program_df.show()

24367
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|                Y|
| RZETLPHZJFYB8|          5|           23|         24|   N

In [16]:
not_vine_program_df = div_df[div_df.verified_purchase == 'N']
print(not_vine_program_df.count())
not_vine_program_df.show()

23441
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R274AKHXHYGIN6|          5|           87|        100|   N|                N|
| R88RERIQDYDT5|          2|           17|         26|   N|                N|
| RBLD20LL9DQYP|          3|           52|         60|   N|                N|
| R82QWN2X2OCHB|          5|          176|        208|   Y|                N|
|R3FMODXVNCITMO|          1|           70|         72|   N|                N|
|R27R4CKOHB9KY5|          1|          241|        254|   N|                N|
|R1KJHW982Z8752|          4|           36|         44|   N|                N|
|R3I0YBN2E2E2K1|          1|           75|         78|   N|                N|
|R37J2T4W2EMTUT|          2|           35|         46|   N|                N|
|R3OD3BMAM5WWNF|          5|           38|         39|   N

In [22]:
five_star_paid = vine_program_df[vine_program_df.star_rating == 5].count()
not_five_star_paid = vine_program_df[vine_program_df.star_rating != 5].count()
percentage = (five_star_paid / not_five_star_paid) * 100
print('Number of 5 stars: ' + str(five_star_paid), 'Percentage: ' + str(round(percentage,2)))

Number of 5 stars: 11742 Percentage: 93.01


In [24]:
five_star_unpaid = not_vine_program_df[not_vine_program_df.star_rating == 5].count()
not_five_star_unpaid = not_vine_program_df[not_vine_program_df.star_rating != 5].count()
percentage = (five_star_unpaid / not_five_star_unpaid) * 100
print('Number of 5 stars: ' + str(five_star_unpaid), 'Percentage: ' + str(round(percentage,2)))

Number of 5 stars: 10453 Percentage: 80.48
